# Data pre-processing and data augmentation

In [ ]:
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [3]:
mkdir preview

In [4]:
ls

Cat_vs_Dog_small.ipynb  data/                   kerashelloworld.ipynb
Iris_sanguinea.JPG      keras_MNIST.ipynb       preview/
WordEmbed.ipynb         keras_MNIST.py


In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

Using Theano backend.


## Prepare the Data

In [10]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## Training a small convnet from scratch

In [6]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
K.set_image_dim_ordering('th')

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

In [7]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 148, 148)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 148, 148)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 74, 74)    0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 72, 72)    9248        maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [14]:
model.fit_generator(
        train_generator,
        samples_per_epoch=20,     #should be 2000, but i just use a sample data set
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=20)         #should be 800
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
20/20 [==============================] - 1s - loss: 0.0348 - acc: 1.0000 - val_loss: 1.9637 - val_acc: 0.5000
Epoch 2/10
20/20 [==============================] - 1s - loss: 0.0289 - acc: 1.0000 - val_loss: 2.3141 - val_acc: 0.4500
Epoch 3/10
20/20 [==============================] - 1s - loss: 0.0473 - acc: 1.0000 - val_loss: 2.9762 - val_acc: 0.5500
Epoch 4/10
20/20 [==============================] - 1s - loss: 0.0470 - acc: 0.9500 - val_loss: 1.9951 - val_acc: 0.6000
Epoch 5/10
20/20 [==============================] - 1s - loss: 0.0109 - acc: 1.0000 - val_loss: 2.1362 - val_acc: 0.6000
Epoch 6/10
20/20 [==============================] - 1s - loss: 0.0053 - acc: 1.0000 - val_loss: 2.2314 - val_acc: 0.6000
Epoch 7/10
20/20 [==============================] - 1s - loss: 0.0360 - acc: 1.0000 - val_loss: 2.2316 - val_acc: 0.5000
Epoch 8/10
20/20 [==============================] - 1s - loss: 0.0929 - acc: 0.9500 - val_loss: 1.9233 - val_acc: 0.6500
Epoch 9/10
20/20 [==============

# Using the bottleneck features of a pre-trained network

In [50]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

# path to the model weights file.
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000   #should be 2000
nb_validation_samples = 800    #should be 800
nb_epoch = 50

In [23]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    # build the VGG16 network
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # load the weights of the VGG16 networks
    # (trained on ImageNet, won the ILSVRC competition in 2014)
    # note: when there is a complete match between your model definition
    # and your weight savefile, you can simply call model.load_weights(filename)
    assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
    f = h5py.File(weights_path)
    layers = f.attrs.values()
    for k in range(len(layers)):
        if k >= len(model.layers):
            # we don't look at the last (fully-connected) layers in the savefile
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    f.close()
    print('Model loaded.')

    
    ## generator data and save features calculated by pre-trained model
    generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)    #change to 'wb'

    generator = datagen.flow_from_directory(
            validation_data_dir,
            target_size=(img_width, img_height),
            batch_size=32,
            class_mode=None,
            shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

* We can try this actually: https://keras.io/applications/#vgg16
* Load a full vgg16, load weights, pop top layers and freeze convnet layers
* calculate feature is slow

In [55]:
'''
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model =  VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(3, img_width, img_height))  

datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
            'data/valid',
            target_size=(img_width, img_height),
            batch_size=20,
            class_mode=None,
            shuffle=False)

features_validation = model.predict_generator(generator, 20)
np.save(open('vgg16feature_validation.npy', 'wb'), features_validation)
'''

Found 40 images belonging to 2 classes.


In [37]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=32,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [39]:
save_bottlebeck_features()

Model loaded.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [38]:
train_top_model()

Train on 20 samples, validate on 20 samples
Epoch 1/50
20/20 [==============================] - 0s - loss: 0.6933 - acc: 0.4000 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/50
20/20 [==============================] - 0s - loss: 0.6913 - acc: 0.5000 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 3/50
20/20 [==============================] - 0s - loss: 0.6887 - acc: 0.8000 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 4/50
20/20 [==============================] - 0s - loss: 0.6904 - acc: 0.5000 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 5/50
20/20 [==============================] - 0s - loss: 0.6952 - acc: 0.5000 - val_loss: 0.6940 - val_acc: 0.5000
Epoch 6/50
20/20 [==============================] - 0s - loss: 0.6859 - acc: 0.7000 - val_loss: 0.6941 - val_acc: 0.5000
Epoch 7/50
20/20 [==============================] - 0s - loss: 0.6859 - acc: 0.7000 - val_loss: 0.6942 - val_acc: 0.5000
Epoch 8/50
20/20 [==============================] - 0s - loss: 0.6937 - acc: 0.5000 - val_loss: 0.6944 - val_

## Fine-tuning the top layers of a a pre-trained network

To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:

* instantiate the convolutional base of VGG16 and load its weights
* add our previously defined fully-connected model on top, and load its weights
* freeze the layers of the VGG16 model up to the last convolutional block

Note that:

* in order to perform fine-tuning, all layers should start with properly trained weights: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it.
* we choose to only fine-tune the last convolutional block rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features).
* fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small, so as not to wreck the previously learned features.

In [ ]:
for layer in model.layers[:25]:
    layer.trainable = False